## Importaciones

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

## Datos

### Cargar

He plasmado todos los datos dentro de archivos CSV, que están en la carpeta ["data"](data/), por lo que ahora para cargarlos sólo hay que crear un DataFrame de Pandas vacío y juntar en él los datos de todos los archivos. Un bucle "for" nos hace el trabajo independientemente de cuántos archivos estén (Siempre y cuando todos tengan la misma estructura claro, pero estoy cuidando eso)

In [5]:
dataset = pd.DataFrame()

for archivo in os.listdir("./data"):
    if archivo.endswith(".csv"):
        dataset = pd.concat([dataset, pd.read_csv(os.path.join("./data", archivo), delimiter=",", quotechar='"', header=None)])

### Procesar

El método para almacenar los datos funciona, pero ello no implica que un modelo pueda entenderlos. Deben ser tokenizados, para ello Keras lleva un tokenizador y nos sirve con los datos así como los he guardado

En éste caso es tan fácil como, inicializar el tokenizador y usar su método `.fit_on_texts()` para ajustar su vocabulario y demás al de los datos

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

tokenizer.fit_on_texts(dataset[0] + dataset[1])

Al estar éste tokenizador ajustado a los datos de entrenamiento, y teniendo en cuenta que se va a usar para cargar todos los datos que el modelo vaya a leer o escribir, es conveniente guardarlo para futuras ejecuciones, de la misma forma que se guarda el modelo. La siguiente celda se encarga de éso

In [ ]:
with open('tokenizer_config.txt', 'w', encoding='utf-8') as f:
    f.write(str(tokenizer.get_config()))

Para cargar el tokenizador exportado se usaría éste código: 
```python
with open('tokenizer_config.txt', 'r', encoding='utf-8') as f:
    tokenizer_config_loaded = f.read()
    tokenizer_config_dict = eval(tokenizer_config_loaded)
    tokenizer_loaded = Tokenizer.from_config(tokenizer_config_dict)
```

In [ ]:
preguntas = tokenizer.texts_to_sequences(dataset[0])
respuestas = tokenizer.texts_to_sequences(dataset[1])

In [ ]:
# Agregar padding a las secuencias
max_len = max([len(seq) for seq in preguntas + respuestas])
preguntas_padded = tf.keras.preprocessing.sequence.pad_sequences(preguntas, maxlen=max_len)
respuestas_padded = tf.keras.preprocessing.sequence.pad_sequences(respuestas, maxlen=max_len)

### Ordenar aleatoriamente

Es una buena práctica otorgarle al modelo los datos ordenados de manera aleatoria, y de hecho por cómo estamos cargando los datos tenemos una ventaja adicional:
- El modelo no aprende patrones raros en función del orden en el que reciba datos
- Al momento de reservar una parte para validación, nos aseguramos de que tanto en entrenamiento como en validación hay datos de todas las categorías

In [ ]:
# Calcular el índice de división
split_idx = int(0.8 * len(preguntas_padded))

# Barajar los índices
indices = np.arange(len(preguntas_padded))
np.random.shuffle(indices)

# Reordenar los datos según los índices barajados
preguntas_padded_shuffled = preguntas_padded[indices]
respuestas_padded_shuffled = respuestas_padded[indices]

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val = np.split(preguntas_padded_shuffled, [split_idx])
y_train, y_val = np.split(respuestas_padded_shuffled, [split_idx])

## Modelo

El estado de los datos tiene buena pinta, toca ir preparando el modelo (Y si no siempre se puede agregar más código arriba)

### Checkpointer

Keras tiene un objeto que se coloca en los callbacks de `.fit()` y sirve para ir guardando el modelo a medida que entrena, lo que nos permite cortar el entrenamiento y continuarlo después o directamente llevar uno de los guardados a prueba manual, y tal vez producción

Asimismo, los argumentos del checkpointer sirven para elegir entre hacer varias cosas, por ejemplo...
- Ir guardando cada copia del modelo poniéndole una variable al nombre
- Que el nombre sea invariable y una copia se sobreescriba sobre la anterior, teniendo siempre 1
- Guardar una copia tras cada etapa
- Guardar una copia tras cada etapa pero sólo si cumple algún requisito, que puede ser haber bajado o subido alguna variable

En éste caso particular la configuración será la siguiente: 
- Sólo se guarda copia si el modelo ha bajado su pérdida de validación, pues es la forma más rápida y sencilla de ver si está mejorando
- Cada copia sobreescribe a la anterior. En teoría, si se está guardando una copia, significa que es como la anterior pero mejor, por lo que para qué conservar la antigua

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint(
    filepath="./checkpoints/model.tf",
    monitor="val_loss",
    mode=min,
    save_best_only=True
)

Éste objeto de Keras se encarga de ir guardando el modelo en mitad del entrenamiento. Al no tener ninguna variable el nombre (aunque se le podrían poner), en la ruta sólo va a haber un modelo siempre, y gracias al resto de parámetros, siempre será el que haya sacado la pérdida de validación (val_loss) más baja

### Arquitectura

Todo el código ha sido escrito por [Phind](https://www.phind.com/) así que ni lo termino de entender ni sé si funcionará, pero a darle

In [ ]:
from keras.layers import Input, LSTM, Dense, Embedding
from keras.models import Model

# Dimensiones de los embeddings y de las unidades LSTM
embedding_dim = 128
lstm_units = 256

# Encoder
encoder_inputs = Input(shape=(max_len,))
encoder_embedding = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_len,))
decoder_embedding = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Modelo
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)